<h1 style="color:#987753"><center>============= IMPORT DEPANDENCIES =============</center></h1>

In [ ]:
import tensorflow as tf
import numpy as np
from random import shuffle
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt
import time
import random
from datetime import datetime
from datetime import date
from random import randint
from tensorflow.keras.callbacks import TensorBoard
import pandas as pd
import csv
import os


<h1 style="color:#987753"><center>============= UTILS FUNCTION =============</center></h1>

In [ ]:
def getTime():
    
    now = datetime.now()
    current_time = now.strftime("%H-%M-%S")
    today = str(date.today()).split('-')
    today = today[2]+"_"+today[1]+"_"+today[0]
    runTime = ("%s_%s" %(today,current_time))
    return runTime


In [ ]:
def logName():
    
    NAME = "ModelCifar10_MLP_"+str(ID)+"_"+getTime()
    return NAME

In [ ]:
def hyperParams():
    
    #model.add(Dense(64, input_dim=64, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)))
    
    modelParams = {"Layers":[{"neurons":64 , "activation":"relu","dropOut":False, "kernel_regularizer":"regularizers.l2(0.0001)", "activity_regularizer":False},
                    {"neurons":46 , "activation":"relu", "dropOut":0.3, "kernel_regularizer":False, "activity_regularizer":False},
                    {"neurons":2500 , "activation":"relu", "dropOut":0.3, "kernel_regularizer":False, "activity_regularizer":False},
                    {"neurons":10, "activation":"softmax", "dropOut":0.1, "kernel_regularizer":False, "activity_regularizer":False}],
                   "loss":"categorical_crossentropy",
                   "optimizer":"Adam(lr=0.001)",
                   "metrics":"accuracy",
                   "learningRate":.001,
                   "epochs":50,
                   "batchSize":500,
                   "testName":"Regu"
                 }
    return modelParams

In [ ]:
def humainesLabels(position):
    labels = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
    return labels[position]

In [ ]:
def humanizeTime(secs):
    mins, secs = divmod(secs, 60)
    hours, mins = divmod(mins, 60)
    return ('RUN TIME : %02d:%02d:%02d' % (hours, mins, secs))

In [ ]:
def loadData():
    
    (trainImages, trainLabels), (testImages, testLabels) = cifar10.load_data()
    trainImages, testImages = trainImages / 255.0, testImages / 255.0
    return trainImages, testImages, trainLabels, testLabels


In [ ]:
def plotFigure(images=False, labels=False, nbSamples=0, size=10):
    
    plt.figure(figsize=(size, size))
    for i in range(nbSamples):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
       
        plt.xlabel(humainesLabels(labels[i][0]))
    plt.show()
    

<h1 style="color:#987753"><center>============= INIT MODEL =============</center></h1>

In [ ]:
def oneHoteEncoding(trainLabels, testLabels):
    
    trainLabels = tf.keras.utils.to_categorical(trainLabels)
    testLabels = tf.keras.utils.to_categorical(testLabels)
    
    return trainLabels, testLabels


In [ ]:
def createModel(params):
    
    model = Sequential()

    model.add(Flatten(input_shape=(32, 32, 3)))
    for layers in params['Layers']:
        
        # ================ If kernel_regu & activity_regu == False ==============
        if (layers['kernel_regularizer'] == False) and (layers['activity_regularizer'] == False):
            model.add(Dense(layers['neurons'], activation=layers['activation']))
        # ============================== End False ==============================
        
        # ================ If kernel_regu & activity_regu True ==============
        elif (layers['kernel_regularizer'] == True) and (layers['activity_regularizer'] == True):
            model.add(Dense(layers['neurons'], activation=layers['activation'], kernel_regularizer=layers['kernel_regularizer'], activity_regularizer=layers['activity_regularizer']))
        
        # ================ If kernel_regu = True ==============
        elif (layers['kernel_regularizer'] == True) and (layers['activity_regularizer'] == False):
            model.add(Dense(layers['neurons'], activation=layers['activation'], kernel_regularizer=layers['kernel_regularizer']))
        
        # ================ If kernel_regu & activity_regu True ==============
        elif (layers['kernel_regularizer'] == False) and (layers['activity_regularizer'] == True):
            model.add(Dense(layers['neurons'], activation=layers['activation'], activity_regularizer=layers['activity_regularizer']))
            
        # ================ If dropout ==============
        if (layers['dropOut'] != False):
            model.add(Dropout(layers['dropOut']))
    
    opt = Adam(lr=0.001, beta_1=0.9)
    model.compile(loss=params['loss'],
                   optimizer=opt,
                   metrics=['accuracy'])
    
    model.summary()
    return model


<h1 style="color:#987753"><center>============= TRAIN MODEL =============</center></h1>

In [ ]:
def reshapeInput(trainImages, testImages):
    
    trainImages.reshape(trainImages.shape[0], trainImages.shape[1] * trainImages.shape[2] * trainImages.shape[3]) #
    testImages.reshape(testImages.shape[0], testImages.shape[1] * testImages.shape[2] * testImages.shape[3]) #
    
    return trainImages, testImages


In [ ]:
def fitModel(model, params, trainImages, trainLabels, testImges, testLabels):
    
    start_time = time.time()
    tensorboard = TensorBoard(log_dir='D:\logsProject\{}'.format(logName()))
    ################################ CALCULATE RUNTIME - START ###############################
    history = model.fit(trainImages, trainLabels,
                       epochs=params['epochs'],
                       batch_size=params['batchSize'], verbose=1, callbacks=[tensorboard], validation_data=(testImges, testLabels))
    ################################ CALCULATE RUNTIME - END ###############################
    print(humanizeTime(time.time() - start_time))
    return model, humanizeTime(time.time() - start_time), history

<h1 style="color:#987753"><center>============= EVALUATE MODEL =============</center></h1>

In [ ]:
def evaluateModel(model, testImages, testLabels):
    
    return model.evaluate(testImages, testLabels, batch_size=50)


<h1 style="color:#987753"><center>============= PREDICT =============</center></h1>

In [ ]:
def predictionPercent(model, testImages, testLabels):
    
    prediction = model.predict(testImages)
    predicted = []
    for i in range(len(prediction)):
        if np.argmax(testLabels[i]) == np.argmax(prediction[i]):
            predicted.append(1)
        else:
            predicted.append(0)
    print("Predicted percent {} %".format((sum(predicted)/(len(predicted)) * 100)))
    result = (sum(predicted)/len(predicted)) * 100
    
    return prediction, result

In [ ]:
def plotPredictedResult(prediction, testImages, testLabels):
    
    plt.figure(figsize=(15, 15))
    for i in range(20):
        plt.subplot(5,7,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(testImages[i])
        plt.xlabel("True: "+humainesLabels(np.argmax(testLabels[i]))+"\n\n\n")
        plt.title("\nPredicted: "+humainesLabels(np.argmax(prediction[i])))
        
    plt.show()

<h1 style="color:#987753"><center>============= SAVE MODEL =============</center></h1>

In [ ]:
def saveModel(model, ID):
    
    now = datetime.now()
    current_time = now.strftime("%H-%M-%S")
    today = str(date.today()).split('-')
    today = today[2]+"_"+today[1]+"_"+today[0]
    runTime = ("%s_%s" %(today,current_time))
    model.save('D:\Models\model_run_'+runTime+"_"+str(ID))


In [ ]:
def readLogFile(path="D:/LogsProject/hyperParams/finalRecap.csv", cols=None):
    
    try:
        
        df = pd.read_csv(path, sep=";" ,encoding='utf-8')
        return df, path
    
    except Exception as e:
        
        print('-Erorr file not found, it will be created now... \n-Error tracback : %s' %(e))
        f = open(path, "w")
        writer = csv.DictWriter(f, fieldnames=cols)
        writer.writeheader()
        f.close()
        df = pd.read_csv(path, sep="," ,encoding='utf-8')
        return df, path

In [ ]:
def generateExcelParams(params, history, resultEV, runTime):
    
    pass

    params['indiv_id'] = ID
    params['Layers'] = ';'.join(str(dict_) for dict_ in params['Layers'])
    params['valAccuracy'] = resultEV[1]
    params['valLoss'] = resultEV[0]
    params['trainAccuracy'] = round(history.history['accuracy'][params['epochs']-1].item(), 3)
    params['trainLoss'] = round(history.history['loss'][params['epochs']-1].item(), 3)
    params['runTime'] = runTime.split(' ')[3]
    
    
    dictToCsv = dict()
    for key, value in zip(params.keys(), params.values()):
        
        tmp = []
        tmp.append(value)
        dictToCsv[key] = tmp
    
    
    df = pd.DataFrame(dictToCsv, columns=params.keys())
    mainDirectory = os.getcwd() + "\\logs\\mlp\\logs_" + getTime()+"\\"
    os.mkdir(mainDirectory)
    df.to_csv(mainDirectory+"combined_recap.csv", sep=";", index=None, header=True)
#     logFile, path = readLogFile(cols=params.keys())
#     dfMerged = pd.concat([logFile, df], ignore_index=True)
#     os.remove(path)
#     dfMerged.to_csv(path, sep=";", index=None, header=True)

#     return dfMerged

In [ ]:
def main():
    global ID
    ID = random.randint(1,1000000)
    print("================================ ID RUN %s ================================" % (ID))
    params = hyperParams()
    (trainImages, testImages, trainLabels, testLabels) = loadData()
    
    print("================================ SHAPE ================================")
    print("\t- NB TRAIN SAMPLES %s WIDTH %s HEIGHT %s COLOR %s" % (trainImages.shape))
    print("\t- NB TEST SAMPLES %s WIDTH %s HEIGHT %s COLOR %s" % (testImages.shape))
    
    print("================================ PLOT FIGURE ================================")
    plotFigure(trainImages, trainLabels, nbSamples=25)
    
    print("================================ ONEHOT ENCODING ================================")
    trainLabels, testLabels = oneHoteEncoding(trainLabels, testLabels)
    
    print("================================ CREATE MODEL ================================")
    model = createModel(params)
    
    print("================================ RESHAPE INPUTS ================================")
    trainImages, testImages = reshapeInput(trainImages, testImages)
    
    print("================================ FIT MODEL ================================")
    model, runTime, history = fitModel(model, params, trainImages, trainLabels, testImages, testLabels)
        
    print("================================ EVALUATE MODEL ================================")
    resultEV = evaluateModel(model, testImages, testLabels)
    
    print("================================ PREDICITION ================================")
    prediction, result = predictionPercent(model, testImages, testLabels)
    
    print("================================ PLOT PREDICITION ================================")
    plotPredictedResult(prediction, testImages, testLabels)
    
    print("================================ SAVE MODEL ================================")
#     saveModel(model, ID)
    
    print("================================ GENERATE CSV HYPERPARAMS ================================")
    jsonDf = generateExcelParams(params, history, resultEV, runTime)
#     jsonDf.head()

In [ ]:
main()